In [2]:
import numpy as np
from quaos.core.circuits.target import find_map_to_target_pauli_sum
from quaos.core.circuits import Gate, SWAP
from quaos.utils import get_linear_dependencies
from quaos.graph_utils import find_one_permutation, mapping_key, brute_force_all_permutations, find_swapped_dependent_elements
from quaos.models import ToricCode, Hadamard_Symmetric_PauliSum, SWAP_symmetric_PauliSum
from quaos.models.random_hamiltonian import random_symmetric_pauli_sum
from scripts.experiments.symmetries.src.pauli import symplectic_pauli_reduction 
from scripts.experiments.symmetries.src.permutations_matroid import find_permutations_matroid

In [3]:
seed = None
 
d = 2
n_qubits = 8
n_sym_q = 2
n_paulis = 18
H, C = Hadamard_Symmetric_PauliSum(n_paulis, n_qubits, n_sym_q, seed=seed)
H.combine_equivalent_paulis()
H.remove_trivial_paulis()


In [4]:
independent_paulis, dependencies = get_linear_dependencies(H.tableau(), d)

print(independent_paulis)
print(dependencies)

[0, 1, 2, 3, 4, 5, 6, 7, 11, 13, 14, 15, 16, 17]
{8: [(0, 1), (1, 1), (6, 1)], 9: [(0, 1), (2, 1)], 10: [(0, 1), (2, 1), (6, 1)], 12: [(0, 1), (3, 1), (6, 1)]}


In [5]:
def permutation_to_swaps(perm: dict[int, int]) -> list[tuple[int, int]]:
    """
    Decompose a permutation (given as dict label->label) into a sequence of swaps (transpositions).
    Each swap is a tuple (i,j) meaning 'swap i and j'.
    Applying these swaps in order reproduces the permutation.

    Parameters
    ----------
    perm : dict[int,int]
        Permutation mapping, e.g. {0:1, 1:2, 2:0, 3:3}.

    Returns
    -------
    swaps : list[tuple[int,int]]
        Sequence of swaps to realize the permutation.
    """
    swaps: list[tuple[int,int]] = []
    seen = set()
    for start in sorted(perm.keys()):
        if start in seen or perm[start] == start:
            continue
        # build the cycle
        cycle = []
        cur = start
        while cur not in seen:
            seen.add(cur)
            cycle.append(cur)
            cur = perm[cur]
        if len(cycle) > 1:
            # decompose cycle (c0 c1 ... c_{k-1}) as (c0 ck-1)(c0 ck-2)...(c0 c1)
            c0 = cycle[0]
            for j in range(len(cycle) - 1, 0, -1):
                swaps.append((c0, cycle[j]))
    return swaps


def build_spm_checker(hamiltonian):
    def spm_checker(permutation):
        pairs = permutation_to_swaps(permutation)
        if pairs == []:
            return False
        H_target = hamiltonian.copy()
        for p in pairs:
            H_target.swap_paulis(p[0], p[1])
        return np.array_equal(H_target.symplectic_product_matrix(), hamiltonian.symplectic_product_matrix())

    return spm_checker


In [6]:
checker = build_spm_checker(H)
perms = find_permutations_matroid(independent_paulis, dependencies, checker, p=2, k=1)[0]
print(perms)

{0: 0, 1: 1, 2: 2, 3: 3, 4: 15, 5: 5, 6: 6, 7: 7, 8: 8, 9: 9, 10: 10, 11: 11, 12: 12, 13: 13, 14: 14, 15: 4, 16: 16, 17: 17}


In [7]:
n_independent = len(independent_paulis)
automorphism = []
for i in independent_paulis:
    automorphism.append(perms[i])

H_t = H.copy()
H_t = H_t[automorphism]
H_i = H[independent_paulis]

print(H_i)
print(H_t)
print(np.array_equal(H_i.symplectic_product_matrix(), H_t.symplectic_product_matrix()))
F, h, _, _ = find_map_to_target_pauli_sum(H_i, H_t)


(1+0j)|x1z0 x0z0 x1z1 x1z0 x1z0 x0z1 x0z1 x1z1 | 0 
(1+0j)|x1z0 x0z0 x1z1 x1z0 x0z1 x1z0 x1z0 x0z1 | 0 
(1+0j)|x1z0 x0z0 x0z1 x1z0 x1z0 x1z0 x0z1 x0z1 | 0 
(1+0j)|x1z0 x0z0 x0z0 x1z0 x1z1 x0z0 x1z0 x0z1 | 0 
(1+0j)|x0z0 x1z0 x0z0 x0z1 x0z0 x1z0 x0z1 x1z0 | 0 
(1+0j)|x0z0 x0z0 x1z0 x1z1 x1z1 x1z1 x1z1 x1z1 | 0 
(1+0j)|x0z1 x0z0 x1z1 x1z0 x1z0 x0z1 x0z1 x1z1 | 0 
(1+0j)|x0z0 x0z0 x1z1 x1z0 x1z0 x0z0 x0z0 x0z1 | 0 
(1+0j)|x0z0 x0z0 x0z0 x1z0 x1z0 x0z0 x1z1 x1z0 | 0 
(1+0j)|x0z0 x0z0 x0z0 x1z1 x1z0 x0z1 x1z0 x0z1 | 0 
(1+0j)|x0z0 x0z0 x0z1 x1z1 x0z0 x1z0 x1z1 x0z0 | 0 
(1+0j)|x0z0 x0z1 x0z0 x0z1 x0z0 x1z0 x0z1 x1z0 | 0 
(1+0j)|x0z0 x0z0 x0z0 x0z0 x0z0 x1z1 x0z1 x0z1 | 0 
(1+0j)|x0z0 x0z0 x0z1 x0z1 x0z1 x0z1 x0z0 x1z1 | 0 

(1+0j)|x1z0 x0z0 x1z1 x1z0 x1z0 x0z1 x0z1 x1z1 | 0 
(1+0j)|x1z0 x0z0 x1z1 x1z0 x0z1 x1z0 x1z0 x0z1 | 0 
(1+0j)|x1z0 x0z0 x0z1 x1z0 x1z0 x1z0 x0z1 x0z1 | 0 
(1+0j)|x1z0 x0z0 x0z0 x1z0 x1z1 x0z0 x1z0 x0z1 | 0 
(1+0j)|x0z0 x0z1 x0z0 x0z1 x0z0 x1z0 x0z1 x1z0 | 0 
(1+0j)|x0z0

In [23]:
G = Gate('Symmetry', [i for i in range(H.n_qudits())], F.T, 2, h)
print(F)


[[1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0]
 [0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0]
 [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]]


In [24]:
# test in loop
n_tests = 5
passed = 0
seed = None
d = 2
for _ in range(n_tests):
    n_qubits = 14
    n_paulis = 30
    n_sym_q = 2
    H, C = Hadamard_Symmetric_PauliSum(n_paulis, n_qubits, n_sym_q, seed=seed)
    H.combine_equivalent_paulis()
    print('Got H')

    checker = build_spm_checker(H)
    independent_paulis, dependencies = get_linear_dependencies(H.tableau(), d)

    perms = find_permutations_matroid(independent_paulis, dependencies, checker, p=2, k=1)
    n_independent = len(independent_paulis)
    automorphism = []
    for i in independent_paulis:
        automorphism.append(perms[0][i])

    H_t = H.copy()
    H_t = H_t[automorphism]
    H_i = H[independent_paulis]

    F, h, _, _ = find_map_to_target_pauli_sum(H_i, H_t)
    G = Gate('Symmetry', [i for i in range(H.n_qudits())], F.T, 2, h)
    
    if G.act(H).standard_form() == H.standard_form() and not np.array_equal(G.symplectic, np.eye(2 * H.n_qudits())):
        print('Got Gate')
        passed += 1
    else:
        print('failed')
        print(independent_paulis, dependencies)
        print(perms)
        print(automorphism)
        print(G.symplectic)
        print(G.act(H).standard_form())
        print(H.standard_form())

print(f'Passed {passed}/{n_tests}')

KeyboardInterrupt: 

In [8]:
# test in loop
n_tests = 5
passed = 0
seed = None
d = 2

sym = SWAP(0, 1, 2)

for _ in range(n_tests):
    n_qubits = 3
    n_paulis = 3
    H = random_symmetric_pauli_sum(sym, n_qubits, n_paulis)
    H.weights = np.ones(n_paulis)
    print('Got H')

    checker = build_spm_checker(H)
    independent_paulis, dependencies = get_linear_dependencies(H.tableau(), d)
    print(H.n_paulis())
    print(independent_paulis, dependencies)
    perms = find_permutations_matroid(independent_paulis, dependencies, checker, p=2, k=1)
    print(perms)
    n_independent = len(independent_paulis)
    automorphism = []
    for i in independent_paulis:
        automorphism.append(perms[0][i])

    H_t = H.copy()
    H_t = H_t[automorphism]
    H_i = H[independent_paulis]

    F, h, _, _ = find_map_to_target_pauli_sum(H_i, H_t)
    G = Gate('Symmetry', [i for i in range(H.n_qudits())], F.T, 2, h)
    
    if G.act(H).standard_form() == H.standard_form() and not np.array_equal(G.symplectic, np.eye(2 * H.n_qudits())):
        print('Got Gate')
        passed += 1
    else:
        print('failed')
        print(independent_paulis, dependencies)
        print(perms)
        print(automorphism)
        print(G.symplectic)
        print(G.act(H).standard_form())
        print(H.standard_form())

3 0


IndexError: index 3 is out of bounds for axis 0 with size 3